In [2]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [5]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(
        env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = \
        filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

C:\Users\AS\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


0: loss=0.702, reward_mean=19.9, rw_bound=22.0
1: loss=0.689, reward_mean=28.1, rw_bound=33.0
2: loss=0.670, reward_mean=38.2, rw_bound=33.5
3: loss=0.664, reward_mean=36.9, rw_bound=45.0
4: loss=0.652, reward_mean=43.1, rw_bound=50.5
5: loss=0.634, reward_mean=34.1, rw_bound=38.0
6: loss=0.632, reward_mean=40.1, rw_bound=53.0
7: loss=0.630, reward_mean=34.8, rw_bound=34.0
8: loss=0.634, reward_mean=53.6, rw_bound=53.0
9: loss=0.608, reward_mean=45.1, rw_bound=52.0
10: loss=0.617, reward_mean=66.9, rw_bound=76.5
11: loss=0.614, reward_mean=71.6, rw_bound=82.5
12: loss=0.614, reward_mean=76.1, rw_bound=88.0
13: loss=0.608, reward_mean=70.3, rw_bound=70.0
14: loss=0.600, reward_mean=87.4, rw_bound=107.5
15: loss=0.602, reward_mean=91.8, rw_bound=124.5
16: loss=0.605, reward_mean=83.7, rw_bound=104.5
17: loss=0.594, reward_mean=95.2, rw_bound=109.5
18: loss=0.597, reward_mean=116.8, rw_bound=135.0
19: loss=0.592, reward_mean=114.9, rw_bound=137.0
20: loss=0.580, reward_mean=138.6, rw_boun